In [2]:
%autosave 1
%matplotlib inline
import urllib.request as req
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random, re
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Autosaving every 1 seconds


In [3]:
# Tensorflow Placeholder

a = tf.placeholder(tf.int32, [None])     # 정수 자료형 3개 배열
b = tf.constant(10);
x10_op = a * b;    # 배열의 모든 값을 10개 하는 연산의 정의

sess = tf.Session()

r1 = sess.run(x10_op, feed_dict={a:[1,2,3,4,5]})
print(r1)
r2 = sess.run(x10_op, feed_dict={a:[10,20]})
print(r2)

[10 20 30 40 50]
[100 200]


In [11]:
# bmi.csv 생성 코드

def calc_bmi(h, w):
    bmi = w / (h/100) ** 2
    if bmi < 18.5: return "thin"
    if bmi < 25: return "normal"
    return "fat"

fp = open("bmi.csv", "w", encoding="utf-8")
fp.write("height,weight,label\r\n")

cnt = {"thin":0, "normal":0, "fat":0}
for i in range(20000):
    h = random.randint(120, 200)
    w = random.randint(35, 80)
    label = calc_bmi(h, w)
    cnt[label] += 1
    fp.write("{0},{1},{2}\r\n".format(h, w, label))
fp.close()
print("ok", cnt)

ok {'thin': 6413, 'normal': 5998, 'fat': 7589}


In [3]:
csv = pd.read_csv("bmi.csv")

#2 데이터 정규화
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100

#3 레이블 -> 배열 (벡터화?)
bclass = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

#4 테스트 데이터 분류
test_csv = csv[15000:20000]
test_pat = test_csv[["weight", "height"]]
test_ans = list(test_csv["label_pat"])

#5 데이터 플로우 그래프 구축 및 플레이스 홀더 선언
x = tf.placeholder(tf.float32, [None, 2])
y_ = tf.placeholder(tf.float32, [None, 3])

W = tf.Variable(tf.zeros([2,3]))    #6 가중치
b = tf.Variable(tf.zeros([3]))      # 바이어스
y = tf.nn.softmax(tf.matmul(x, W) + b)  #7 소프트맥스(Wx+b) 회귀 정의

#8 모델 훈련
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.005)
train = optimizer.minimize(cross_entropy)

# 정답률
predict = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

# 세션 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(3500):
    i = (step * 100) % 14000
    rows = csv[1+i : 1+i+100]
    x_pat = rows[["weight", "height"]]
    y_ans = list(rows["label_pat"])
    fd = {x:x_pat, y_:y_ans}
    sess.run(train, feed_dict=fd)
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)
        acc = sess.run(accuracy, feed_dict={x:test_pat, y_:test_ans})
        print("step=", step, "cre=", cre, "acc=", acc)
    
# 최종 정답
acc = sess.run(accuracy, feed_dict={x:test_pat, y_:test_ans})
print("score=", acc)

step= 0 cre= 109.492 acc= 0.3756
step= 500 cre= 67.8577 acc= 0.8346
step= 1000 cre= 52.653 acc= 0.8894
step= 1500 cre= 48.1174 acc= 0.8866
step= 2000 cre= 42.2109 acc= 0.9204
step= 2500 cre= 43.0748 acc= 0.9408
step= 3000 cre= 38.7978 acc= 0.9478
score= 0.9506


In [4]:
# 시각화?

a = tf.constant(20, name="a")
b = tf.constant(30, name="b")
mul_op = a * b

sess = tf.Session()

tw = tf.summary.FileWriter("log_dir", graph=sess.graph)

print(sess.run(mul_op))
summary, _ = sess.run()

600
